In [1]:
import cadquery as cq
from jupyter_cadquery import set_defaults, set_sidecar, get_defaults, reset_cache, Animation
from jupyter_cadquery.cadquery import show, PartGroup, Part, Faces, Edges
from jupyter_cadquery.mate_assembly import Mate, MAssembly

set_sidecar("Hexapod")
set_defaults(axes=True, axes0=True, edge_accuracy=0.01, mate_scale=5, zoom=3, timeit=True)

Overwriting auto display for cadquery Workplane and Shape


In [2]:
from jupyter_cadquery.cad_renderer import RENDER_CACHE

# Hexapod 

![2-hexapod.png](2-hexapod.png)

In [3]:
import numpy as np
horizontal_angle = 25

def intervals(count):
    r = [ min(180, (90 + i*(360 // count)) % 360) for i in range(count)]
    return r 

def times(end, count):
    return np.linspace(0, end, count+1)
    
def vertical(count, end, offset, reverse):
    factor = -1 if reverse else 1
    ints = intervals(count)
    heights = [factor * round(35 * np.sin(np.deg2rad(x)) - 15, 1) for x in ints]
    heights.append(heights[0])
    return times(end, count), heights[offset:] + heights[1:offset+1]

def horizontal(end, reverse):
    factor = -1 if reverse else 1
    return times(end, 4), [0, factor * horizontal_angle, 0, -factor * horizontal_angle, 0]

print("Leg group 1 (transparent)")
print("horizontal movement    ", horizontal(4, True))
print("vertical heights (left) ", vertical(8, 4, 0, True))
print("vertical heights (right)", vertical(8, 4, 0, False))

print("\nLeg group 1 (filled)")
print("horizontal movement", horizontal(4, False))
print("vertical heights (left) ", vertical(8, 4, 4, True))
print("vertical heights (right)", vertical(8, 4, 4, False))


Leg group 1 (transparent)
horizontal movement     (array([0., 1., 2., 3., 4.]), [0, -25, 0, 25, 0])
vertical heights (left)  (array([0. , 0.5, 1. , 1.5, 2. , 2.5, 3. , 3.5, 4. ]), [-20.0, -9.7, 15.0, 15.0, 15.0, 15.0, 15.0, -9.7, -20.0])
vertical heights (right) (array([0. , 0.5, 1. , 1.5, 2. , 2.5, 3. , 3.5, 4. ]), [20.0, 9.7, -15.0, -15.0, -15.0, -15.0, -15.0, 9.7, 20.0])

Leg group 1 (filled)
horizontal movement (array([0., 1., 2., 3., 4.]), [0, 25, 0, -25, 0])
vertical heights (left)  (array([0. , 0.5, 1. , 1.5, 2. , 2.5, 3. , 3.5, 4. ]), [15.0, 15.0, 15.0, -9.7, -20.0, -9.7, 15.0, 15.0, 15.0])
vertical heights (right) (array([0. , 0.5, 1. , 1.5, 2. , 2.5, 3. , 3.5, 4. ]), [-15.0, -15.0, -15.0, 9.7, 20.0, 9.7, -15.0, -15.0, -15.0])


# PartGroup

## Parts

In [4]:
thickness = 2
height = 40
width = 65
length = 100
diam = 4
tol = 0.05

In [5]:
x1, x2 = 0.63, 0.87
base_holes = {
    "right_back": (-x1*length, -x1*width), "right_middle": (0, -x2*width), "right_front": ( x1*length, -x1*width),
    "left_back":  (-x1*length,  x1*width), "left_middle":  (0,  x2*width), "left_front":  ( x1*length,  x1*width),
}
stand_dist = {"front_stand": 0.75 * length, "back_stand": -0.8 * length}

base = (cq.Workplane()
    .ellipse(length, width).pushPoints(list(base_holes.values())).circle(diam / 2 + tol)
    .extrude(thickness)
)

stand_cutout = cq.Workplane().box(thickness + 2 * tol, width / 2 + 2 * tol, thickness * 2)
front_cutout = cq.Workplane().box(length / 3, 2 * (width + tol), 2 * thickness)

base = (base
    .cut(front_cutout.translate((length, 0, 0)))
    .cut(stand_cutout.translate((stand_dist["back_stand"], 0, thickness)))
    .cut(stand_cutout.translate((stand_dist["front_stand"], 0, thickness)))
    .faces(">X").edges("not |Y").fillet(width / 5)
)

In [6]:
stand = cq.Workplane().box(height, width / 2 + 10, thickness)
inset = cq.Workplane().box(thickness, width / 2, thickness)
backing = cq.Workplane("ZX").polyline([(10,0), (0,0), (0, 10)]).close().extrude(thickness)

stand = (stand
    .union(inset.translate(( (height + thickness) / 2, 0, 0)))
    .union(inset.translate((-(height + thickness) / 2, 0, 0)))
    .union(backing.translate((-height / 2, -thickness / 2, thickness / 2)))
    .union(backing.rotate((0, 0, 0), (0, 1, 0), -90).translate((height / 2, -thickness / 2, thickness / 2)))
)


In [7]:
l1, l2 = 50, 80
pts = [( 0,  0), ( 0, height/2), (l1, height/2 - 5), (l2, 0)]
upper_leg_hole = (l2 - 10, 0)

upper_leg = (cq.Workplane()
    .polyline(pts).mirrorX()
    .pushPoints([upper_leg_hole]).circle(diam/2 + tol).extrude(thickness)
    .edges("|Z and (not <X)").fillet(4)
)

axle = (cq.Workplane("XZ", origin=(0, height/2 + thickness + tol, thickness/2))
    .circle(diam/2).extrude(2 * (height/2 + thickness + tol))
)

upper_leg = upper_leg.union(axle)

In [8]:
w, l1, l2 = 15, 20, 120
pts = [( 0,  0), ( l1, w), (l2, 0)]
lower_leg_hole = (l1 - 10, 0)

lower_leg = (cq.Workplane()
    .polyline(pts).mirrorX()
    .pushPoints([lower_leg_hole]).circle(diam/2 + tol)
    .extrude(thickness)
    .edges("|Z").fillet(5)
)

In [9]:
leg_angles = {
    "right_back": 105,  "right_middle": 90, "right_front": 75, 
    "left_back": -105,  "left_middle": -90, "left_front": -75,
}
leg_names = list(leg_angles.keys())
stand_names = ("front_stand", "back_stand")

In [11]:
reset_cache()
show(
    base, 
    stand.translate((0,100,thickness/2)), 
    upper_leg, # .translate((-100,-100,0)),
    lower_leg, # .translate((0,-100,0)),
    parallel=True
)

Using multiprocessing


DEBUG:root: Tessellate object(hash=1396799925)
DEBUG:root: Tessellate object(hash=1396150005)
DEBUG:root: Tessellate object(hash=1395951317)
DEBUG:root: Tessellate object(hash=1395688997)


T 100.07810592651367
T 133.2693099975586
T 138.9000415802002
T 291.149377822876
parallel tessellation time: 570 ms
   - build mesh time: 13 ms
   - discretize time: 3 ms
         - edge list: 8 ms
render shape Group                          time: 27 ms
   - build mesh time: 12 ms
   - discretize time: 0 ms
         - edge list: 5 ms
render shape Group                          time: 20 ms
   - build mesh time: 12 ms
   - discretize time: 0 ms
         - edge list: 7 ms
render shape Group                          time: 21 ms
   - build mesh time: 12 ms
   - discretize time: 0 ms
         - edge list: 6 ms
render shape Group                          time: 20 ms
 overall render time: 679 ms
Done, using side car 'Hexapod'


## Define PartGroup

In [11]:
def create_hexapod():
    # Some shortcuts
    L = lambda *args: cq.Location(cq.Vector(*args))
    C = lambda *args: cq.Color(*args)

    # Leg assembly
    leg = (MAssembly(upper_leg, name="upper", color=C("orange"))
        .add(lower_leg, name="lower", color=C("orange"), loc=L(80,0,0))
    )
    # Hexapod assembly
    hexapod = (MAssembly(base, name="bottom", color=C("gray"), loc=L(0, 1.1*width, 0))
        .add(base, name="top", color=C(0.9, 0.9, 0.9), loc=L(0, -2.2*width, 0))
        .add(stand, name="front_stand", color=C(0.5, 0.8, 0.9), loc=L( 40, 100, 0))
        .add(stand, name="back_stand", color=C(0.5, 0.8, 0.9), loc=L(-40, 100, 0))
    )
    for i, name in enumerate(leg_names):
        hexapod.add(leg, name=name, loc=L(100, -55*(i-1.7), 0))

    return hexapod

## Define Mates 

In [12]:
#reset_cache()

In [21]:
hexapod = create_hexapod()

M = lambda selector, *obj_selectors: Mate(hexapod.find(selector, *obj_selectors))

hexapod.mate(
    name="bottom", selector="bottom", origin=True,
    mate=M("bottom", "faces@>Z")
).mate(
    name="top", selector="top", origin=True, 
    mate=M("top", "faces@<Z").rx(180).tz(-(height + 2 * tol))
)

angle = {"front_stand":180, "back_stand":0}
for name in stand_names:
    hexapod.mate(
        name=f"bottom_{name}", selector="bottom", 
        mate=M("bottom", "faces@<Z", ("wires", (stand_dist[name], 0))).rz(angle[name])
    ).mate(
        name=f"{name}_lower", selector=name, origin=True,
        mate=M(name, "faces@<X").rz(90)
    )

for name, pnt in base_holes.items():
    hexapod.mate(
        name=f"bottom_{name}", selector="bottom", 
        mate=M("bottom", "faces@<Z", ("wires", pnt)).rz(leg_angles[name])
    )

for name in leg_names:
    lower, upper, angle = (">Z", "<Z", 75) if "left" in name else ("<Z", ">Z", -75)
    hexapod.mate(
        name=f"{name}_lower_hinge",  selector=f"{name}>lower", origin=True, 
        mate=M(f"{name}>lower", ("faces", lower), ("wires", lower_leg_hole)).rx(180)
    ).mate(
        name=f"{name}_front_hinge",  selector=name,
        mate=M(name, ("faces", upper), ("wires", upper_leg_hole)).rz(angle)
    ).mate(
        name=f"{name}_bottom_hinge", selector=name, origin=True,
        mate=M(name, "faces@<Y")
    )
    
reset_cache()
show(hexapod, render_mates=True, axes=False, timeit=True, parallel=True)

Using multiprocessing


DEBUG:root: Tessellate object(hash=606042726)
DEBUG:root: Tessellate object(hash=605213318)
DEBUG:root: Tessellate object(hash=605959990)
DEBUG:root: Tessellate object(hash=605790118)


parallel tessellation time: 501 ms
   - build mesh time: 18 ms
   - discretize time: 6 ms
         - edge list: 12 ms
render shape bottom                         time: 38 ms
   - discretize time: 0 ms
         - edge list: 7 ms
render shape mates                          time: 7 ms
   - discretize time: 0 ms
         - edge list: 7 ms
render shape mates                          time: 7 ms
   - discretize time: 0 ms
         - edge list: 7 ms
render shape mates                          time: 7 ms
   - discretize time: 0 ms
         - edge list: 7 ms
render shape mates                          time: 7 ms
   - discretize time: 0 ms
         - edge list: 7 ms
render shape mates                          time: 7 ms
   - discretize time: 0 ms
         - edge list: 7 ms
render shape mates                          time: 7 ms
   - discretize time: 0 ms
         - edge list: 7 ms
render shape mates                          time: 7 ms
   - discretize time: 0 ms
         - edge list: 7 ms
render sh

## Relocate and assemble

In [14]:
set_defaults(parallel=True)

In [45]:
reset_cache()

In [ ]:
get_defaults()

In [60]:
hexapod.relocate()
reset_cache()

show(hexapod, render_mates=False, parallel=True)

parallel True
using multiprocessing


DEBUG:root: Tessellate object(hash=1320792153)
DEBUG:root: Tessellate object(hash=1457769673)
DEBUG:root: Tessellate object(hash=1209859273)
DEBUG:root: Tessellate object(hash=1335426137)
DEBUG:root: Tessellate object(hash=1316933561)
DEBUG:root: Tessellate object(hash=1225630921)
DEBUG:root: Tessellate object(hash=1225680985)
DEBUG:root: Tessellate object(hash=1247420617)
DEBUG:root: Tessellate object(hash=1459183817)
DEBUG:root: Tessellate object(hash=135509081)
DEBUG:root: Tessellate object(hash=1210647353)
DEBUG:root: Tessellate object(hash=1225136329)
DEBUG:root: Tessellate object(hash=1321861945)
DEBUG:root: Tessellate object(hash=1240884313)
DEBUG:root: Tessellate object(hash=1208467545)
DEBUG:root: Tessellate object(hash=1233857465)


parallel tessellation time: 776 ms
   - build mesh time: 19 ms
   - discretize time: 6 ms
         - edge list: 12 ms
render shape bottom                         time: 41 ms
   - build mesh time: 17 ms
   - discretize time: 5 ms
         - edge list: 11 ms
render shape top                            time: 36 ms
   - build mesh time: 17 ms
   - discretize time: 1 ms
         - edge list: 7 ms
render shape front_stand                    time: 28 ms
   - build mesh time: 17 ms
   - discretize time: 1 ms
         - edge list: 7 ms
render shape back_stand                     time: 27 ms
   - build mesh time: 17 ms
   - discretize time: 1 ms
         - edge list: 8 ms
render shape right_back                     time: 28 ms
   - build mesh time: 17 ms
   - discretize time: 1 ms
         - edge list: 8 ms
render shape lower                          time: 28 ms
   - build mesh time: 17 ms
   - discretize time: 1 ms
         - edge list: 8 ms
render shape right_middle                   time: 28 

In [ ]:
get_default("parallel")

In [ ]:
from jupyter_cadquery.cad_renderer import RENDER_CACHE
RENDER_CACHE.set_log_level("DEBUG")

In [ ]:
reset_cache()

In [ ]:
for leg in leg_names:
    hexapod.assemble(f"{leg}_bottom_hinge", f"bottom_{leg}")
    hexapod.assemble(f"{leg}_lower_hinge", f"{leg}_front_hinge")

for stand_name in stand_names:
    hexapod.assemble(f"{stand_name}_lower", f"bottom_{stand_name}")

hexapod.assemble("top", "bottom")

d = show(hexapod, render_mates=False, grid=True, axes=False, parallel=True)

## Animation

In [ ]:
leg_group = ("left_front", "right_middle", "left_back")

animation = Animation(d.root_group)

for name in leg_names:
    # move upper leg
    animation.add_number_track(f"bottom>{name}", "rz", *horizontal(4, "middle" in name))
    
    # move lower leg
    animation.add_number_track(f"bottom>{name}>lower", "rz", *vertical(8, 4, 0 if name in leg_group else 4, "left" in name))
    
    # lift hexapod to run on grid
    animation.add_number_track(f"bottom", "tz", [0, 4], [61.25]*2)
    
animation.animate(speed=3)